##### IBM DATA SCIENCE NEIGHBORHOOD SEGMENTATION

In [1]:
# @hidden_cell
credentials = {
    'IAM_SERVICE_ID': 'iam-ServiceId-d0f8f1bd-e2f0-406a-8332-37fd6aa968ed',
    'IBM_API_KEY_ID': 'i5BveX5kkRvPQYvpNHUpCk5i94fy6VpFpDlF34h2iTXZ',
    'ENDPOINT': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'IBM_AUTH_ENDPOINT': 'https://iam.ng.bluemix.net/oidc/token',
    'BUCKET': 'project-donotdelete-pr-8ka5t6nq466jou',
    'FILE': 'canada_postal_codes.csv'
}

CLIENT_ID = 'XUTFTOO2ER5MV2YMGAMZ34XFGEEMJ2QN3ZPBREVDGCY1DBK0'
CLIENT_SECRET = 'NFFPP4M0INLKO30VNSS3ZJYVKY0S1Z2R0ZSFBN4OVEYFIAJU'
VERSION = '20190812'

In [2]:
from ibm_botocore.client import Config
import ibm_boto3

def download_file(credentials):  
    cos = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials['IBM_API_KEY_ID'],
    ibm_service_instance_id=credentials['IAM_SERVICE_ID'],
    ibm_auth_endpoint=credentials['IBM_AUTH_ENDPOINT'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials['ENDPOINT'])
    
    try:
        res = cos.download_file(Bucket=credentials['BUCKET'], Key=credentials['FILE'], Filename=credentials['FILE'])
        print('SUCCESSFULLY DOWNLOADED FILE')
    except Exception:
        print('FAILED TO DOWNLOAD FILE')

In [3]:
import pandas as pd
import numpy as np

##### PART 1: DOWNLOAD/CLEAN DATA

In [4]:
download_file(credentials)
df = pd.read_csv(credentials['FILE'])
df.head(10)

SUCCESSFULLY DOWNLOADED FILE


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


In [5]:
download_file(credentials)
df = pd.read_csv(credentials['FILE'])
df.rename(columns={'Postcode': 'Postal Code', 'Neighbourhood': 'Neighborhood'}, inplace=True)
df = df[df.Borough != 'Not assigned']
df.Neighborhood.replace('Not assigned', df.Borough, inplace=True)
df = df.groupby(['Postal Code', 'Borough'], sort=False)['Neighborhood'].apply(', '.join).reset_index()
df.sort_values('Postal Code', inplace=True)
df.index = list(range(len(df)))
df.head(10)

SUCCESSFULLY DOWNLOADED FILE


,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [6]:
print('ENTRIES IN DATAFRAME:', len(df))

ENTRIES IN DATAFRAME: 103


##### PART 2: CONCATENATE COORDINATES TO DATA

In [7]:
coord_path = 'https://cocl.us/Geospatial_data'
df_coord = pd.read_csv(coord_path)
df['Latitude'] = df_coord['Latitude']
df['Longitude'] = df_coord['Longitude']
df.head(10)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


##### PART 3: VISUALIZE NEIGHBORHOODS IN TORONTO

In [8]:
df = df[df['Borough'].str.contains('Toronto')]
df.index = list(range(len(df)))
df.head(10)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
7,M4S,Central Toronto,Davisville,43.704324,-79.388790
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
9,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049


In [9]:
print('ENTRIES IN DATAFRAME:', len(df))

ENTRIES IN DATAFRAME: 38


In [10]:
!pip install folium
import folium

In [11]:
lat = 43.6532
lng = -79.3832
toronto_map = folium.Map(location=[lat, lng], zoom_start=12)
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label)
    folium.CircleMarker([lat, lng], radius=5, popup=label, color='blue', fill=True, fill_color='#3186CC', fill_opacity=0.8).add_to(toronto_map)
toronto_map

##### PART 4: EXPLORE NEIGHBORHOODS IN TORONTO

In [12]:
import requests
import json

In [13]:
RADIUS = 500
LIMIT = 100

def get_venues(name, lat, lng):
    venues = []
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, RADIUS, LIMIT)
    results = requests.get(url).json()['response']['groups'][0]['items']
    for v in results:
        info = [name, lat, lng, 
                v['venue']['name'], v['venue']['location']['lat'], v['venue']['location']['lng'], v['venue']['categories'][0]['name']]
        venues.append(info)
    df = pd.DataFrame(venues)
    df.columns = ['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude', 
                         'Venue', 'Venue Latitude', 'Venue Longitude', 'Category']
    return df

In [14]:
venue_df = pd.DataFrame()
for name, lat, lng in zip(df['Neighborhood'], df['Latitude'], df['Longitude']):
    venues = get_venues(name, lat, lng)
    venue_df = pd.concat([venue_df, venues])
venue_df.reset_index(inplace=True, drop=True)
venue_df.head(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Glen Stewart Ravine,43.676300,-79.294784,Other Great Outdoors
4,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
5,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant
6,"The Danforth West, Riverdale",43.679557,-79.352188,Dolce Gelato,43.677773,-79.351187,Ice Cream Shop
7,"The Danforth West, Riverdale",43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop
8,"The Danforth West, Riverdale",43.679557,-79.352188,Cafe Fiorentina,43.677743,-79.350115,Italian Restaurant
9,"The Danforth West, Riverdale",43.679557,-79.352188,La Diperie,43.677530,-79.352295,Ice Cream Shop


In [15]:
print('ENTRIES IN DATAFRAME:', len(venue_df))

ENTRIES IN DATAFRAME: 1690


In [16]:
venue_df.groupby('Neighborhood').count()['Venue']

Neighborhood
Adelaide, King, Richmond                                                                                      100
Berczy Park                                                                                                    56
Brockton, Exhibition Place, Parkdale Village                                                                   23
Business Reply Mail Processing Centre 969 Eastern                                                              16
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara     16
Cabbagetown, St. James Town                                                                                    46
Central Bay Street                                                                                             82
Chinatown, Grange Park, Kensington Market                                                                     100
Christie                                                                   

In [17]:
print('NUMBER OF UNIQUE CATEGORIES:', len(venue_df['Category'].unique()))

NUMBER OF UNIQUE CATEGORIES: 232


##### PART 5: ANALYZE NEIGHBORHOODS IN TORONTO

In [18]:
onehot_df = pd.get_dummies(venue_df[['Category']], prefix='', prefix_sep='')
onehot_df['Neighborhood'] = venue_df['Neighborhood']
new_columns = [onehot_df.columns[-1]] + list(onehot_df.columns[:-1])
onehot_df = onehot_df[new_columns]
onehot_df.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
print('ENTRIES IN DATAFRAME:', len(onehot_df))

ENTRIES IN DATAFRAME: 1690


In [20]:
grouped_df = onehot_df.groupby('Neighborhood').mean().reset_index()
grouped_df.head(10)

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,"Adelaide, King, Richmond",0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.030000,...,0.000000,0.00,0.0,0.0,0.0,0.010000,0.000000,0.000000,0.010000,0.000000
1,Berczy Park,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.000000,0.00,0.0,0.0,0.0,0.017857,0.000000,0.000000,0.000000,0.000000
2,"Brockton, Exhibition Place, Parkdale Village",0.043478,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.000000,0.00,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
3,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.000000,0.00,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.000000,0.000000,0.0625,0.0625,0.0625,0.125,0.1875,0.125,0.000000,...,0.000000,0.00,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
5,"Cabbagetown, St. James Town",0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.021739,...,0.000000,0.00,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
6,Central Bay Street,0.012195,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.012195,...,0.000000,0.00,0.0,0.0,0.0,0.012195,0.000000,0.000000,0.012195,0.000000
7,"Chinatown, Grange Park, Kensington Market",0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.000000,0.01,0.0,0.0,0.0,0.060000,0.000000,0.040000,0.010000,0.000000
8,Christie,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.000000,0.00,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
9,Church and Wellesley,0.012195,0.012195,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.012195,...,0.012195,0.00,0.0,0.0,0.0,0.000000,0.012195,0.012195,0.000000,0.012195


In [21]:
print('ENTRIES IN DATAFRAME:', len(grouped_df))

ENTRIES IN DATAFRAME: 38


In [22]:
NUM_TOP_VENUES = 5

def top_venues(row):
    categories = row.iloc[1:]
    sorted_categories = categories.sort_values(ascending=False)
    return sorted_categories.index.values[0:NUM_TOP_VENUES]

In [23]:
columns = ['Neighborhood']
for i in range(NUM_TOP_VENUES):
    columns.append('Top Venue {}'.format(i+1))
venues_sorted_df = pd.DataFrame(columns=columns)
venues_sorted_df['Neighborhood'] = grouped_df['Neighborhood']

for i in range(len(grouped_df)):
    venues_sorted_df.iloc[i, 1:] = top_venues(grouped_df.iloc[i, :])
venues_sorted_df.head(10)

,Neighborhood,Top Venue 1,Top Venue 2,Top Venue 3,Top Venue 4,Top Venue 5
0,"Adelaide, King, Richmond",Coffee Shop,Café,Thai Restaurant,Bar,Steakhouse
1,Berczy Park,Coffee Shop,Cocktail Bar,Café,Beer Bar,Bakery
2,"Brockton, Exhibition Place, Parkdale Village",Breakfast Spot,Café,Coffee Shop,Yoga Studio,Gym
3,Business Reply Mail Processing Centre 969 Eastern,Gym / Fitness Center,Auto Workshop,Park,Pizza Place,Restaurant
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Service,Airport Terminal,Airport Lounge,Harbor / Marina,Coffee Shop
5,"Cabbagetown, St. James Town",Restaurant,Coffee Shop,Bakery,Flower Shop,Pub
6,Central Bay Street,Coffee Shop,Café,Ice Cream Shop,Italian Restaurant,Sandwich Place
7,"Chinatown, Grange Park, Kensington Market",Café,Vegetarian / Vegan Restaurant,Chinese Restaurant,Bar,Mexican Restaurant
8,Christie,Grocery Store,Café,Park,Coffee Shop,Baby Store
9,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Gay Bar


##### PART 6: CLUSTER NEIGHBORHOODS IN TORONTO

In [24]:
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

In [25]:
K = 5
clustering_df = grouped_df.drop('Neighborhood', axis=1)
kmeans = KMeans(n_clusters=K, random_state=0).fit(clustering_df)
kmeans.labels_[0:NUM_TOP_VENUES]

array([0, 0, 0, 0, 0], dtype=int32)

In [26]:
venues_sorted_df.insert(0, 'Cluster', kmeans.labels_)
merged_df = df
merged_df = merged_df.join(venues_sorted_df.set_index('Neighborhood'), on='Neighborhood')
merged_df.head(10)

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster,Top Venue 1,Top Venue 2,Top Venue 3,Top Venue 4,Top Venue 5
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Other Great Outdoors,Trail,Pub,Discount Store
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Furniture / Home Store
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,0,Pet Store,Italian Restaurant,Steakhouse,Sushi Restaurant,Brewery
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,Bakery,Italian Restaurant,Gastropub
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,4,Photography Studio,Park,Bus Line,Swim School,Dive Bar
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197,0,Gym / Fitness Center,Hotel,Clothing Store,Sandwich Place,Breakfast Spot
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,0,Coffee Shop,Clothing Store,Sporting Goods Shop,Yoga Studio,Bagel Shop
7,M4S,Central Toronto,Davisville,43.704324,-79.388790,0,Pizza Place,Dessert Shop,Sandwich Place,Italian Restaurant,Café
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,1,Playground,Trail,Summer Camp,Wings Joint,Discount Store
9,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049,0,Coffee Shop,Pub,American Restaurant,Liquor Store,Sports Bar


In [27]:
x = np.arange(K)
ys = [i + x + (i*x)**2 for i in range(K)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

cluster_map = folium.Map(location=[lat, lng], zoom_start=12)
for lat, lng, neighborhood, cluster in zip(merged_df['Latitude'], merged_df['Longitude'], merged_df['Neighborhood'], merged_df['Cluster']):
    label = '{}, CLUSTER: {}'.format(neighborhood, cluster)
    label = folium.Popup(label)
    folium.CircleMarker([lat, lng], radius=5, popup=label, 
                        color=rainbow[cluster-1], fill=True, fill_color=rainbow[cluster-1], fill_opacity=0.8).add_to(cluster_map)
cluster_map